## RAG_Clova

#### Env

Python Version : Python 3.12.2

requirements : 
https://ssl.pstatic.net/static/clova/service/hyperclova/cookbook/rag/requirements.txt

In [1]:
# Vector DB인 Milvus와 관련된 모듈들은 모듈의 용도를 명확히 구분하기 위해 Vector DB 구축 단계에서 불러왔으며 해당 부분에서 코드를 확인하실 수 있습니다.
import json
import os
import subprocess
from langchain_community.document_loaders import UnstructuredHTMLLoader
from pathlib import Path
import base64
import http.client
from tqdm import tqdm
import requests
import dotenv
dotenv.load_dotenv()

True

#### 1. Raw Data → Connecting

In [2]:
url_to_filename_map = {}
wget_path="/opt/homebrew/bin/wget"
with open("clovastudiourl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]
 
folder_path = "clovastudioguide"
 
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for url in urls:
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    subprocess.run([wget_path, "-O", file_path, url], check=True)
    url_to_filename_map[url] = filename
 
with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file)

--2024-09-15 23:30:21--  https://guide.ncloud-docs.com/docs/clovastudio-info
Resolving guide.ncloud-docs.com (guide.ncloud-docs.com)... 104.18.7.159, 104.18.6.159
Connecting to guide.ncloud-docs.com (guide.ncloud-docs.com)|104.18.7.159|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-09-15 23:30:22 ERROR 403: Forbidden.



CalledProcessError: Command '['/opt/homebrew/bin/wget', '-O', 'clovastudioguide/clovastudio-info.html', 'https://guide.ncloud-docs.com/docs/clovastudio-info']' returned non-zero exit status 8.

In [5]:
import os
import json
import requests
from tqdm import tqdm

url_to_filename_map = {}
folder_path = "clovastudioguide"

with open("clovastudiourl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

for url in tqdm(urls, desc="Downloading"):
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        with open(file_path, "wb") as file:
            file.write(response.content)
        
        url_to_filename_map[url] = filename

    except requests.RequestException as e:
        print(f"An error occurred while downloading {url}: {e}")

with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file, indent=4)

print("Download complete. Mapping saved to url_to_filename_map.json")


Downloading: 100%|██████████| 7/7 [00:06<00:00,  1.14it/s]

Download complete. Mapping saved to url_to_filename_map.json


##### Quote

txt 파일을 작성할 때는 한 줄에 하나의 URL을 작성해야 합니다. 이때, 각 URL은 줄바꿈으로 구분되어야 합니다.

그리고 html 데이터를 로딩하기 전, robots.txt를 통해 데이터로 활용할 사이트의 접근 허용 여부를 확인해야합니다. NCP의 클로바스튜디오 사용 가이드의 경우 User-agent: * Allow:/ 로 접근이 가능합니다.

예제에서 활용한 HTML은, 네이버클라우드 플랫폼(NCP)의 클로바스튜디오 사용 가이드중 CLOVA Studio("AI Services" → "CLOVA Studio")와 관련된 모든 안내 페이지를 데이터로 활용했습니다.

LangChain을 활용해 로딩한 html은, 파일을 저장한 디렉토리의 주소를 metadata의 'source'로 가져오게 됩니다. 추후 답변 제공시, 디렉토리 주소가 아닌 실제 URL을 제공하기 위해, LangChain이 로딩한 데이터를 수정해야합니다.

html을 로딩할 때 파일명을 URL로 할 경우, /와 : 기호로 인해 파일명이 깨지게 됩니다. 따라서 원본 URL과 로딩한 HTML 파일을 쌍으로 mapping하고, 이 정보를 json 형식으로 "url_to_filename_map"에 저장합니다.

이후, html 파일의 저장 경로가 담긴 'source'를, 이 json 파일과 연결해 실제 URL로 변환해줍니다. 위 코드는, json 파일을 저장하는 단계까지입니다.

##### LangChain 활용 HTML 로딩



In [6]:
# 폴더 이름에 맞게 수정
html_files_dir = Path('/Users/minu/dev/Naver/enjoyAll-AI/clovastudioguide')
 
html_files = list(html_files_dir.glob("*.html"))
 
clovastudiodatas = []
 
for html_file in html_files:
    loader = UnstructuredHTMLLoader(str(html_file))
    document_data = loader.load()
    clovastudiodatas.append(document_data)
    print(f"Processed {html_file}")

In [20]:
print(len(clovastudiodatas))
if len(clovastudiodatas) > 0:
    print(clovastudiodatas[0])

0


##### Quote

각 HTML에는 page_content에 모든 텍스트가, metadata의 'source'에는 저장 경로가 기록되어 있습니다. 위는, 로딩된 데이터 중 하나인 clovastudio-info.html의 형태입니다. metadata의 'source'에 실제 URL이 아닌 html 파일이 저장된 디렉토리 경로가 담긴 것을 확인할 수 있습니다. Mapping을 통해 이 'source'를 실제 URL로 바꾸는 작업을 다음 단계에 진행하게 됩니다.

##### Mapping 정보를 활용해 'source'를 실제 URL로 대체

In [19]:
with open("url_to_filename_map.json", "r") as map_file:
    url_to_filename_map = json.load(map_file)
 
filename_to_url_map = {v: k for k, v in url_to_filename_map.items()}

# langchainwikidatas 리스트의 각 Document 객체의 'source' 수정
for doc_list in clovastudiodatas:
    for doc in doc_list:
        extracted_filename = doc.metadata["source"].split("/")[-1]
        print(doc.metadata["source"].split("/"))
        if extracted_filename in filename_to_url_map:
            doc.metadata["source"] = filename_to_url_map[extracted_filename]
        else:
            print(f"Warning: {extracted_filename}에 해당하는 URL을 찾을 수 없습니다.")

In [8]:
# 이중 리스트를 풀어서 하나의 리스트로 만드는 작업
clovastudiodatas_flattened = [item for sublist in clovastudiodatas for item in sublist]

In [9]:
clovastudiodatas_flattened 

[]

#### 2. Chunking



임베딩 모델이 처리할 수 있는 적당한 크기로 raw data를 나누는 것은 매우 중요합니다. 이는 임베딩 모델마다 한 번에 처리할 수 있는 토큰 수의 한계가 있기 때문입니다. CLOVA Studio의 문단 나누기 API는 모델이 직접 문장들간의 의미 유사도를 찾아 최적의 chunk 개수와 사용자가 원하는 1개 chunk의 크기(글자 수)를 직접 설정하여 문단을 나눌 수도 있습니다. 추가로, 후처리(postProcess = True)를 통해 chunk당 글자 수의 상한선과 하한선을 postProcessMaxSize와 postProcessMinSize로 조절할 수도 있습니다.

In [18]:
class SegmentationExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def _send_request(self, completion_request):
        headers = {
            "Content-Type": "application/json; charset=utf-8",
            "X-NCP-CLOVASTUDIO-API-KEY": "NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/",
            "X-NCP-APIGW-API-KEY": "J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB",
            "X-NCP-CLOVASTUDIO-REQUEST-ID": "8147d8eb-a1ff-46e7-8b7e-2c5e6907deb1"
        }
        conn = http.client.HTTPSConnection(self._host)
        print("ht2")
        conn.request(
            "POST",
            "/testapp/v1/api-tools/segmentation/fd948489735b452e90565d7dd15094bd", # If using Service App, change 'testapp' to 'serviceapp', and corresponding app id.
            json.dumps(completion_request),
            headers
        )
        print("ht2")
        response = conn.getresponse()
        print(response.length)
        result = json.loads(response.read().decode(encoding="utf-8"))
        conn.close()
        return result
 
    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res["status"]["code"] == "20000":
            return res["result"]["topicSeg"]
        else:
            raise ValueError(f"{res}")
 
 
if __name__ == "__main__":
    segmentation_executor = SegmentationExecutor(
        host="clovastudio.apigw.ntruss.com",
        api_key="NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/",
        api_key_primary_val="J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB",
        request_id="8147d8eb-a1ff-46e7-8b7e-2c5e6907deb1"
    )
 
    chunked_html = []
 
    for htmldata in tqdm( clovastudiodatas_flattened):
        try:
            request_data = {
                "postProcessMaxSize": 100,
                "alpha": 1.5,
                "segCnt": -1,
                "postProcessMinSize": 0,
                "text": htmldata.page_content,
                "postProcess": False
            }

            request_json_string = json.dumps(request_data)
            request_data = json.loads(request_json_string, strict=False)
            print(1)
            response_data = segmentation_executor.execute(request_data)
        except Exception as e:
            print(f"Error occurred. Message: {e}")
        
        for paragraph in response_data:
            chunked_document = {
                "source": htmldata.metadata["source"],
                "text": paragraph
            }
            chunked_html.append(chunked_document)
 
print(len(chunked_html))

0it [00:00, ?it/s]

0


In [14]:
len(chunked_html)

0

#### 3. Embedding

문단 나누기 API를 통해 나누어진 525개의 chunk(chunked_html)을 CLOVA Studio의 임베딩 API를 사용해 1024차원의 벡터로 변환하는 과정입니다. clir-emb-dolphin의 경우, 임베딩 과정에서 유사도 판단을 위해 벡터의 내적(Inner Product, IP)을 거리 단위로 사용합니다. 반면, clir-sts-dolphin은 코사인 거리(Cosine)를 거리 단위로 사용합니다. 이후, 벡터의 인덱싱과 검색 과정에서 사용자는 어떤 거리 단위를 사용하여 데이터와 사용자의 쿼리 간 유사도를 판단할 것인지 선택할 수 있습니다.

거리 단위를 임베딩부터 인덱싱, 검색까지 일치시켜야 데이터의 품질이 향상되므로, 임베딩 과정에서 사용한 모델(emb / sts)을 기억하는 것이 중요합니다. 임베딩 클래스에 오류가 발생시 멈추게끔 하는 로직을 일부 추가해, 다량의 데이터를 처리할 때 오류 발생시 재실행의 부담을 줄입니다.

In [12]:
class EmbeddingExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id
 
    def _send_request(self, completion_request):
        headers = {
            "Content-Type": "application/json; charset=utf-8",
            "X-NCP-CLOVASTUDIO-API-KEY": self._api_key,
            "X-NCP-APIGW-API-KEY": self._api_key_primary_val,
            "X-NCP-CLOVASTUDIO-REQUEST-ID": self._request_id
        }
 
        conn = http.client.HTTPSConnection(self._host)
        conn.request(
            "POST",
            "/testapp/v1/api-tools/embedding/v2/1d709412831147ecb213c7d0e3f3d373", # If using Service App, change 'testapp' to 'serviceapp', and corresponding app id.
            json.dumps(completion_request),
            headers
        )
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding="utf-8"))
        conn.close()
        return result
 
    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res["status"]["code"] == "20000":
            return res["result"]["embedding"]
        else:
            error_code = res["status"]["code"]
            error_message = res.get("status", {}).get("message", "Unknown error")
            raise ValueError(f"오류 발생: {error_code}: {error_message}")
 
 
if __name__ == "__main__":
    embedding_executor = EmbeddingExecutor(
        host="clovastudio.stream.ntruss.com",
        api_key='NTA0MjU2MWZlZTcxNDJiY9olvu8DfBGDWQ20qeVZrRupbxCgWcs/59xl7cuRCJQ/',
        api_key_primary_val='J97Kg6j6Nrc14yhuSv1JfvgWlJ2qbcOKUsAS88BB',
        request_id='d2d9b51b-aed7-4cf5-9a32-6f101d053352'
    )
 
    for i, chunked_document in enumerate(tqdm(chunked_html)):
        try:
            request_json = {
                "text": chunked_document['text']
            }
            request_json_string = json.dumps(request_json)
            request_data = json.loads(request_json_string, strict=False)
            response_data = embedding_executor.execute(request_data)
        except ValueError as e:
            print(f"Embedding API Error. {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")
         
        chunked_document["embedding"] = response_data

  1%|          | 6/814 [00:00<00:30, 26.76it/s]

Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter


  1%|▏         | 12/814 [00:00<00:29, 27.08it/s]

Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter


  2%|▏         | 18/814 [00:00<00:29, 26.98it/s]

...

Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter


 99%|█████████▉| 807/814 [00:33<00:00, 25.87it/s]

Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter


100%|██████████| 814/814 [00:33<00:00, 24.24it/s]

Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter
Embedding API Error. 오류 발생: 40001: Invalid parameter


In [13]:
dimension_set = set()
 
for item in chunked_html:
    if "embedding" in item:
        dimension = len(item["embedding"])
        dimension_set.add(dimension)
 
print("임베딩된 벡터들의 차원:", dimension_set)

임베딩된 벡터들의 차원: {110}


#### 4. Vector DB

CLOVA Studio의 임베딩 API를 활용하여 모든 chunked_의 text를 1024차원의 벡터로 변환한 후, 'embedding'이라는 이름의 객체로 chunked_html에 추가해 주었습니다. 이제 이 데이터를 Vector DB에 저장하고, 인덱싱을 하는 단계입니다. Vector DB로는 Milvus를 사용하였으며, 대시보드는 Docker를 사용하여 DB를 켜고(Run) 끄는(Stop) 작업을 수행하였습니다.